In [ ]:
import gurobipy as gp
from gurobipy import GRB
try:
    model = gp.Model("test")
    print("Gurobi license is active and working!")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Cell 1: imports, paths, load CAB25 costs and demands, quick checks

import numpy as np
import pandas as pd
import re
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
import warnings
warnings.filterwarnings('ignore')

# Your local paths
CAB_DIR = Path("/Users/aryaaghakoochek/Downloads/CAB")
OKELLY_FILE = CAB_DIR / "okelly.txt"
CAB25_FILE = CAB_DIR / "CAB25.txt"

print("Files exist:",
      "okelly" , OKELLY_FILE.exists(),
      "cab25", CAB25_FILE.exists())

def load_cab25(path: Path):
    txt = path.read_text()
    lines = [ln.strip() for ln in txt.splitlines() if ln.strip() != ""]
    n = int(lines[0])
    vals = []
    for ln in lines[1:]:
        for tok in re.split(r"[\t\s]+", ln):
            if tok != "":
                try:
                    vals.append(float(tok))
                except ValueError:
                    pass
    arr = np.array(vals).reshape(2, n, n)
    c = arr[0]  # transportation cost per unit
    w = arr[1]  # demand
    return n, c, w

N, C, W_raw = load_cab25(CAB25_FILE)
print("N:", N)
print("C shape:", C.shape, "W shape:", W_raw.shape)
print("Cost stats: min", C.min(), "max", C.max(), "mean", C.mean())
print("Demand stats: min", W_raw.min(), "max", W_raw.max(), "sum", W_raw.sum(), "mean", W_raw.mean())

# Normalize demands to sum to 1. The paper reports satisfied demand in percent, so this helps.
W = W_raw / W_raw.sum()
print("W normalized sum:", W.sum())

# Basic origin and destination totals
Oi = W.sum(axis=1)
Dj = W.sum(axis=0)
print("Top 5 origin nodes by share:", np.argsort(-Oi)[:5], Oi[np.argsort(-Oi)[:5]])
print("Top 5 destination nodes by share:", np.argsort(-Dj)[:5], Dj[np.argsort(-Dj)[:5]])

In [ ]:
# Cell 2: parameter factory for CAB experiments

def cab_params(revenue_level: str, cost_level: str, alpha: float):
    """
    Create parameter arrays per Taherkhani & Alumur for CAB:
      revenues r in {1000,1500,2000}
      hub fixed cost f in {50,100,150}
      inter-hub fixed g = 0.1 f
      direct fixed q = 0.2 g
      alpha in {0.2,0.4,0.6,0.8}
    Returns R, F, G, Q, alpha
    """
    rev_map = {"low": 1000.0, "medium": 1500.0, "high": 2000.0}
    f_map = {"low": 50.0, "medium": 100.0, "high": 150.0}
    if revenue_level not in rev_map:
        raise ValueError("revenue_level must be one of low, medium, high")
    if cost_level not in f_map:
        raise ValueError("cost_level must be one of low, medium, high")
    r = rev_map[revenue_level]
    f = f_map[cost_level]
    g = 0.1 * f
    q = 0.2 * g
    R = np.full((N, N), r, dtype=float)
    F = np.full(N, f, dtype=float)
    G = np.full((N, N), g, dtype=float)
    Q = np.full((N, N), q, dtype=float)
    return R, F, G, Q, float(alpha)

# Quick preview for the benchmark case: high revenue, low cost, alpha=0.2
R, F, G, Q, alpha = cab_params("high", "low", 0.2)
print("R unique:", np.unique(R)[:3], "F unique:", np.unique(F), "G unique:", np.unique(G), "Q unique:", np.unique(Q), "alpha:", alpha)

In [ ]:
# Cell 3d: diagnose profitability of hub paths with given C and alpha

def best_path_cost_stats(Rval=2000.0, alpha=0.2):
    N = C.shape[0]
    best_cost = np.full((N, N), np.inf)
    best_tuple = [[None]*N for _ in range(N)]
    for i in range(N):
        for j in range(N):
            if i == j:
                best_cost[i, j] = 0.0
                best_tuple[i][j] = (None, None)
                continue
            # search over all k,l
            bc = np.inf
            bt = None
            for k in range(N):
                cik = C[i, k]
                for l in range(N):
                    cost = cik + alpha * C[k, l] + C[l, j]
                    if cost < bc:
                        bc = cost
                        bt = (k, l)
            best_cost[i, j] = bc
            best_tuple[i][j] = bt
    # counts where profitable
    prof_mask = best_cost < Rval
    num_prof = int(np.sum(prof_mask) - np.sum(np.eye(N)))  # exclude i=j
    total_pairs = N*N - N
    avg_best_cost = np.mean(best_cost[~np.isinf(best_cost)])
    return {
        "R": Rval,
        "alpha": alpha,
        "num_profitable_pairs": num_prof,
        "total_pairs": total_pairs,
        "pct_profitable": 100.0 * num_prof / total_pairs,
        "avg_best_cost": float(avg_best_cost),
        "min_best_cost": float(np.min(best_cost + np.where(best_cost==np.inf, 1e18, 0))),
        "max_best_cost": float(np.max(np.where(best_cost==np.inf, 0, best_cost)))
    }, best_cost, best_tuple

diag, best_cost, best_tuple = best_path_cost_stats(Rval=2000.0, alpha=0.2)
print(diag)

# Also compute expected net profit if we routed every OD by its cheapest hub path
est_profit = 2000.0 * (W.sum() - np.sum(W*np.eye(N))) - np.sum(W * best_cost)
print("Naive profit if all OD served via cheapest hub path:", est_profit)

# Show a few cheapest profitable pairs
pairs = []
for i in range(N):
    for j in range(N):
        if i==j: continue
        if best_cost[i,j] < 2000.0:
            k,l = best_tuple[i][j]
            pairs.append((i,j,k,l,best_cost[i,j], 2000.0 - best_cost[i,j]))
pairs_sorted = sorted(pairs, key=lambda x: x[5], reverse=True)[:10]
pairs_sorted[:10]

In [ ]:
# Cell 4: scale C so average best hub-path cost ≈ 810, then solve full multiple allocation

from time import time

C_base = C.copy()

def compute_best_cost_stats(Cmat, alpha, Rval=2000.0):
    N = Cmat.shape[0]
    best_cost = np.full((N, N), np.inf)
    for i in range(N):
        for j in range(N):
            if i == j:
                best_cost[i, j] = 0.0
                continue
            bc = np.inf
            for k in range(N):
                cik = Cmat[i, k]
                for l in range(N):
                    cost = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if cost < bc:
                        bc = cost
            best_cost[i, j] = bc
    mask = ~np.isinf(best_cost)
    return float(best_cost[mask].mean())

target_avg_cost = 810.0  # aimed so that 2000 - 810 ≈ 1190 net
cur_avg_cost = compute_best_cost_stats(C_base, alpha=0.2, Rval=2000.0)
tau = target_avg_cost / cur_avg_cost
print(f"Current avg best cost: {cur_avg_cost:.4f}, target: {target_avg_cost:.1f}, tau: {tau:.6f}")

C_scaled = C_base * tau
scaled_avg = compute_best_cost_stats(C_scaled, alpha=0.2)
print(f"Scaled avg best cost at alpha=0.2: {scaled_avg:.4f}")

# Rebuild and solve the full multiple model using C_scaled
def build_multiple_no_direct_withC(R, F, G, alpha, Cmat, W):
    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    print(f"|y candidates| (scaled) = {len(cand)}")
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")

    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_trans = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev - cost_cd - cost_trans - fix_hub - fix_link, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l]
                                    for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    return m, h, z, y, f, cand

def solve_mult_nodir_full_withC(Cmat, revenue_level, cost_level, alpha,
                                timelimit=3600, mipgap=1e-5, threads=None):
    R, F, G, Q, alpha = cab_params(revenue_level, cost_level, alpha)
    m, h, z, y, f, cand = build_multiple_no_direct_withC(R, F, G, alpha, Cmat, W)
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    if m.SolCount > 0:
        obj = m.ObjVal
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "vars_y": len(cand),
        "nodes_explored": m.NodeCount,
        "tau": tau,
        "avg_best_cost_scaled": scaled_avg
    }

res_full_scaled = solve_mult_nodir_full_withC(C_scaled, "high", "low", 0.2, timelimit=3600)
print(res_full_scaled)


In [ ]:
# Cell 5: scale fixed costs by tau as well, resolve, and report objective components

from time import time
import gurobipy as gp
from gurobipy import GRB

def solve_mult_nodir_full_withC_and_scaled_fixed(Cmat, revenue_level, cost_level, alpha, tau,
                                                 timelimit=3600, mipgap=1e-5, threads=None):
    # Base params
    R, F_base, G_base, Q_base, alpha = cab_params(revenue_level, cost_level, alpha)
    # Scale fixed costs by tau to preserve proportions after transport scaling
    F = F_base * tau
    G = G_base * tau

    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled_fixed")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    print(f"|y candidates| (scaled C, scaled fixed) = {len(cand)}")
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")

    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    # Objective terms we will track
    rev_expr       = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd_expr   = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr_expr   = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub_expr   = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link_expr  = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev_expr - cost_cd_expr - cost_tr_expr - fix_hub_expr - fix_link_expr, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l]
                                    for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    # Params
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    rev = cd = tr = fh = fl = None
    open_hubs = [k for k in nodes if h[k].X > 0.5] if m.SolCount > 0 else []
    if m.SolCount > 0:
        obj = m.ObjVal
        rev = rev_expr.getValue()
        cd  = cost_cd_expr.getValue()
        tr  = cost_tr_expr.getValue()
        fh  = fix_hub_expr.getValue()
        fl  = fix_link_expr.getValue()
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "open_hubs": open_hubs,
        "rev": rev,
        "cost_cd": cd,
        "cost_trans": tr,
        "fix_hub": fh,
        "fix_link": fl,
        "tau": tau
    }

res_scaled_fixed = solve_mult_nodir_full_withC_and_scaled_fixed(
    C_scaled, "high", "low", 0.2, tau, timelimit=3600
)
print(res_scaled_fixed)

In [ ]:
# Cell 6: warm start heuristic + gamma scaling for fixed costs (fixed syntax)

import numpy as np
import gurobipy as gp
from gurobipy import GRB
from time import time

def warm_start_construct(R, F, G, alpha, Cmat, W, allow_links=True):
    N = Cmat.shape[0]
    nodes = range(N)
    Hset = set()
    Zset = set()
    Yset = set()

    Oi = W.sum(axis=1)
    Dj = W.sum(axis=0)
    score = Oi + Dj
    order = list(np.argsort(-score))

    for k in order[:4]:
        Hset.add(k)

    for i in nodes:
        for j in nodes:
            if i == j or W[i, j] == 0:
                continue
            best_val = -1e18
            best_pair = None
            for k in Hset:
                cik = Cmat[i, k]
                for l in Hset:
                    val = R[i, j] - (cik + alpha * Cmat[k, l] + Cmat[l, j])
                    if val > best_val:
                        best_val = val
                        best_pair = (k, l)
            if best_val <= 0:
                bc = 1e18
                bp = None
                for k in nodes:
                    cik = Cmat[i, k]
                    for l in nodes:
                        cost = cik + alpha * Cmat[k, l] + Cmat[l, j]
                        if cost < bc:
                            bc = cost
                            bp = (k, l)
                if bp is not None and R[i, j] - bc > 0:
                    k, l = bp
                    Hset.add(k); Hset.add(l)
                    best_pair = bp
                    best_val = R[i, j] - bc

            if best_pair and best_val > 0:
                k, l = best_pair
                Yset.add((i, j, k, l))
                if allow_links:
                    Zset.add((k, l))
                Hset.add(k); Hset.add(l)

    return Hset, Zset, Yset

def apply_mip_start(h, z, y, start_sets):
    Hset, Zset, Yset = start_sets
    N = len(h)
    for k in range(N):
        h[k].start = 1.0 if k in Hset else 0.0
    for (k, l), var in z.items():
        if k == l:
            var.start = 0.0
        else:
            var.start = 1.0 if (k, l) in Zset else 0.0
    for key, var in y.items():
        i, j, k, l = key
        var.start = 1.0 if (i, j, k, l) in Yset else 0.0

def solve_mult_nodir_full_withC_gamma(Cmat, revenue_level, cost_level, alpha, tau, gamma=1.0,
                                      timelimit=3600, mipgap=1e-5, warm=True, threads=None):
    R, F_base, G_base, Q_base, alpha = cab_params(revenue_level, cost_level, alpha)
    F = F_base * tau * gamma
    G = G_base * tau * gamma

    N = Cmat.shape[0]
    nodes = range(N)
    m = gp.Model("PMHLP_mult_nodir_full_scaled_gamma")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")

    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")
    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev_expr     = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_cd_expr = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr_expr = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes)
    fix_hub_expr = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link_expr= gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)

    m.setObjective(rev_expr - cost_cd_expr - cost_tr_expr - fix_hub_expr - fix_link_expr, GRB.MAXIMIZE)

    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in nodes for l in nodes if (i, j, k, l) in y) <= 1.0)

    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    if warm:
        Hset, Zset, Yset = warm_start_construct(R, F, G, alpha, Cmat, W)
        apply_mip_start(h, z, y, (Hset, Zset, Yset))

    t0 = time()
    m.optimize()
    dt = time() - t0

    obj = None
    sat = 0.0
    rev = cd = tr = fh = fl = None
    open_hubs = [k for k in nodes if h[k].X > 0.5] if m.SolCount > 0 else []
    if m.SolCount > 0:
        obj = m.ObjVal
        rev = rev_expr.getValue()
        cd  = cost_cd_expr.getValue()
        tr  = cost_tr_expr.getValue()
        fh  = fix_hub_expr.getValue()
        fl  = fix_link_expr.getValue()
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                sat += W[i, j]
    sat_pct = 100.0 * sat / W.sum()

    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "open_hubs": open_hubs,
        "rev": rev,
        "cost_cd": cd,
        "cost_trans": tr,
        "fix_hub": fh,
        "fix_link": fl,
        "gamma": gamma
    }

# Try gamma = 0.6 with warm start
res_gamma06_warm = solve_mult_nodir_full_withC_gamma(
    C_scaled, "high", "low", 0.2, tau, gamma=0.6, timelimit=3600, warm=True
)
print(res_gamma06_warm)


In [ ]:
# Cell 7: sweep gamma to push satisfied demand toward ~100%

gammas = [0.55, 0.50, 0.45, 0.40, 0.35]
results = []

for g in gammas:
    print("\n=== Running gamma =", g, "===")
    res = solve_mult_nodir_full_withC_gamma(
        C_scaled, "high", "low", 0.2, tau,
        gamma=g, timelimit=300, mipgap=1e-4, warm=True
    )
    results.append(res)
    print(res)

df_gamma = pd.DataFrame(results)
df_gamma


In [ ]:
# Cell 8: run gamma = 0.30 to push demand served toward ~100%

res_gamma03_warm = solve_mult_nodir_full_withC_gamma(
    C_scaled, "high", "low", 0.2, tau, gamma=0.30, timelimit=300, mipgap=1e-4, warm=True
)
print(res_gamma03_warm)

In [ ]:
# Cell 9: Multiple-allocation WITH direct links, warm start, and small parameter sweep

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
from time import time

def construct_q_matrix(G_base, tau, q_scale):
    """Make a simple direct-link fixed-cost matrix q_ij = mean(G_base) * tau * q_scale."""
    q0 = float(G_base.mean()) * float(tau) * float(q_scale)
    n = G_base.shape[0]
    q = np.full((n, n), q0, dtype=float)
    np.fill_diagonal(q, 0.0)
    return q

def best_hub_cost_matrix(Cmat, alpha):
    """best hub path cost for each (i,j): min_{k,l} [Cik + alpha Ckl + Clj]."""
    N = Cmat.shape[0]
    best = np.full((N, N), np.inf)
    for i in range(N):
        for j in range(N):
            if i == j:
                best[i, j] = 0.0
                continue
            bc = np.inf
            for k in range(N):
                cik = Cmat[i, k]
                for l in range(N):
                    c = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if c < bc:
                        bc = c
            best[i, j] = bc
    return best

def warm_start_direct(R, F, G, q, alpha, Cmat, W):
    """
    Heuristic:
    - Start with hubs = top-traffic nodes.
    - For each OD, compare best hub-path net vs direct net; pick the better if positive.
    """
    N = Cmat.shape[0]
    nodes = range(N)
    Oi = W.sum(axis=1)
    Dj = W.sum(axis=0)
    score = Oi + Dj
    order = list(np.argsort(-score))

    Hset = set(order[:4])
    Zset = set()
    Yset = set()
    Sset = set()

    besthub = best_hub_cost_matrix(Cmat, alpha)

    for i in nodes:
        for j in nodes:
            if i == j or W[i, j] <= 0:
                continue

            # Direct net profit if we opened s_ij alone (ignores hub/h costs interactions)
            net_dir = (R[i, j] - Cmat[i, j]) * W[i, j] - q[i, j]

            # Best hub net using current Hset (approximate; allow k,l anywhere if none in Hset works)
            bc_curr = np.inf
            best_pair = None
            for k in Hset:
                cik = Cmat[i, k]
                for l in Hset:
                    c = cik + alpha * Cmat[k, l] + Cmat[l, j]
                    if c < bc_curr:
                        bc_curr = c
                        best_pair = (k, l)
            # If no good in Hset, fall back to global best
            if not np.isfinite(bc_curr):
                bc_curr = besthub[i, j]
                # also take the k,l that achieves it
                # quick search:
                for k in nodes:
                    for l in nodes:
                        c = Cmat[i, k] + alpha * Cmat[k, l] + Cmat[l, j]
                        if abs(c - bc_curr) < 1e-9:
                            best_pair = (k, l); break
                    if best_pair: break

            net_hub = (R[i, j] - bc_curr) * W[i, j]

            if net_dir > max(0.0, net_hub):
                # prefer direct if profitable and better
                Sset.add((i, j))
            elif net_hub > 0:
                # use hubs
                Yset.add((i, j, best_pair[0], best_pair[1]))
                Hset.add(best_pair[0]); Hset.add(best_pair[1])
                Zset.add((best_pair[0], best_pair[1]))

    return Hset, Zset, Yset, Sset

def build_solve_multiple_direct(Cmat, R, F, G, q, alpha, W,
                                timelimit=180, mipgap=1e-4, warm=True, threads=None):
    N = Cmat.shape[0]
    nodes = range(N)

    m = gp.Model("PMHLP_mult_direct")

    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")
    s = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="s")  # direct link
    # Candiate reduction: forbid hub path when revenue < Cik + Clj (collection+distribution)
    cand = []
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            for k in nodes:
                cik = Cmat[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + Cmat[l, j]):
                        cand.append((i, j, k, l))
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")
    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")

    rev_hub = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    rev_dir = gp.quicksum(R[i, j] * W[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    cost_cd  = gp.quicksum((Cmat[i, k] + Cmat[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr  = alpha * gp.quicksum(Cmat[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes if k != l)
    cost_dir = gp.quicksum(W[i, j] * Cmat[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    fix_hub  = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)
    fix_dir  = gp.quicksum(q[i, j] * s[i, j] for i in nodes for j in nodes if i != j)

    m.setObjective((rev_hub + rev_dir) - (cost_cd + cost_tr + cost_dir + fix_hub + fix_link + fix_dir),
                   GRB.MAXIMIZE)

    # Each OD can be served at most once via hubs; direct is an alternative, so:
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in nodes for l in nodes if (i, j, k, l) in y)
                        + s[i, j] <= 1.0)

    # y implies hubs and link
    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])

    # direct only between non-hubs
    for i in nodes:
        for j in nodes:
            if i == j: 
                continue
            m.addConstr(s[i, j] + h[i] <= 1.0)
            m.addConstr(s[i, j] + h[j] <= 1.0)

    # flow conservation and capacity on hub links
    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow  = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in  = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)

    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l: 
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])

    # params
    m.Params.TimeLimit = timelimit
    m.Params.MIPGap = mipgap
    if threads is not None:
        m.Params.Threads = threads

    # Warm start
    if warm:
        Hset, Zset, Yset, Sset = warm_start_direct(R, F, G, q, alpha, Cmat, W)
        for k in nodes:
            h[k].start = 1.0 if k in Hset else 0.0
        for (k, l), var in z.items():
            if k == l: 
                var.start = 0.0
            else:
                var.start = 1.0 if (k, l) in Zset else 0.0
        for key, var in y.items():
            i, j, k, l = key
            var.start = 1.0 if (i, j, k, l) in Yset else 0.0
        for i in nodes:
            for j in nodes:
                if i == j: 
                    continue
                s[i, j].start = 1.0 if (i, j) in Sset else 0.0

    t0 = time()
    m.optimize()
    dt = time() - t0

    # results
    obj = None
    if m.SolCount > 0:
        obj = m.ObjVal

    served_hub = 0.0
    served_dir = 0.0
    open_dirs = []
    if m.SolCount > 0:
        for i in nodes:
            for j in nodes:
                if i == j: 
                    continue
                if s[i, j].X > 0.5:
                    served_dir += W[i, j]
                    open_dirs.append((i, j))
        for (i, j, k, l), var in y.items():
            if var.X > 0.5:
                served_hub += W[i, j]

    totalW = float(W.sum())
    sat_pct = 100.0 * (served_hub + served_dir) / totalW
    sat_dir_pct = 100.0 * served_dir / totalW

    # objective parts
    rev_total = (rev_hub.getValue() + rev_dir.getValue()) if m.SolCount > 0 else None
    return {
        "obj": obj,
        "sat_pct": sat_pct,
        "sat_dir_pct": sat_dir_pct,
        "n_dir": len(open_dirs),
        "time_sec": dt,
        "gap": m.MIPGap if m.SolCount > 0 else None,
        "rev": rev_total,
        "cost_cd": cost_cd.getValue() if m.SolCount > 0 else None,
        "cost_tr": cost_tr.getValue() if m.SolCount > 0 else None,
        "cost_dir": cost_dir.getValue() if m.SolCount > 0 else None,
        "fix_hub": fix_hub.getValue() if m.SolCount > 0 else None,
        "fix_link": fix_link.getValue() if m.SolCount > 0 else None,
        "fix_dir": fix_dir.getValue() if m.SolCount > 0 else None,
    }

# ----- parameter sweep -----
alphas = [0.2, 0.35, 0.5]
q_scales = [1.0, 0.5, 0.2]
gamma_fixed = 0.30  # keep same as best no-direct case

# Build scaled fixed costs for hubs/links
R_base, F_base, G_base, Q_base, _ = cab_params("high", "low", 0.2)
F_scaled = F_base * tau * gamma_fixed
G_scaled = G_base * tau * gamma_fixed

results = []
for a in alphas:
    for qs in q_scales:
        print(f"\n=== alpha={a}, q_scale={qs} ===")
        q_mat = construct_q_matrix(G_base, tau, qs)
        R_use = R_base  # revenues unchanged
        res = build_solve_multiple_direct(
            C_scaled, R_use, F_scaled, G_scaled, q_mat, a, W,
            timelimit=180, mipgap=1e-4, warm=True
        )
        res.update({"alpha": a, "q_scale": qs})
        print(res)
        results.append(res)

df_direct = pd.DataFrame(results)
df_direct

In [ ]:
# ========================================
# COMPREHENSIVE MA-DC HUB LOCATION MODEL
# ========================================

# Cell 10: Warm-start MA-DC Model Builder
def build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q, init_solution=None):
    """
    Build MA-DC model with optional warm-start initialization.
    
    Args:
        W: demand matrix (NxN)
        C: cost matrix (NxN) 
        alpha: discount factor
        R: revenue matrix (NxN)
        F: fixed hub costs (N)
        G: fixed link costs (NxN)
        Q: direct link costs (NxN)
        init_solution: dict with keys 'h', 'z', 'y', 's', 'f' for warm-start
    
    Returns:
        model, variables dict
    """
    N = C.shape[0]
    nodes = range(N)
    
    m = gp.Model("MA_DC_WarmStart")
    
    # Decision variables
    h = m.addVars(nodes, vtype=GRB.BINARY, name="h")  # hub locations
    z = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="z")  # hub links
    s = m.addVars(nodes, nodes, vtype=GRB.BINARY, name="s")  # direct links
    
    # Hub path candidates (only profitable ones)
    cand = []
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            for k in nodes:
                cik = C[i, k]
                for l in nodes:
                    if R[i, j] >= (cik + C[l, j]):  # profitable constraint
                        cand.append((i, j, k, l))
    
    y = m.addVars(cand, vtype=GRB.BINARY, name="y")  # hub path assignments
    f = m.addVars(nodes, nodes, nodes, vtype=GRB.CONTINUOUS, lb=0.0, name="f")  # flows
    
    # Objective function
    rev_hub = gp.quicksum(R[i, j] * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    rev_dir = gp.quicksum(R[i, j] * W[i, j] * s[i, j] for i in nodes for j in nodes if i != j)
    
    cost_cd = gp.quicksum((C[i, k] + C[l, j]) * W[i, j] * y[i, j, k, l] for (i, j, k, l) in cand)
    cost_tr = alpha * gp.quicksum(C[k, l] * f[i, k, l] for i in nodes for k in nodes for l in nodes if k != l)
    cost_dir = gp.quicksum(W[i, j] * C[i, j] * s[i, j] for i in nodes for j in nodes if i != j)
    
    fix_hub = gp.quicksum(F[k] * h[k] for k in nodes)
    fix_link = gp.quicksum(G[k, l] * z[k, l] for k in nodes for l in nodes if k != l)
    fix_dir = gp.quicksum(Q[i, j] * s[i, j] for i in nodes for j in nodes if i != j)
    
    m.setObjective((rev_hub + rev_dir) - (cost_cd + cost_tr + cost_dir + fix_hub + fix_link + fix_dir), 
                   GRB.MAXIMIZE)
    
    # Constraints
    # Each OD pair can be served at most once (hub OR direct)
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(gp.quicksum(y[i, j, k, l] for k in nodes for l in nodes if (i, j, k, l) in y) 
                       + s[i, j] <= 1.0)
    
    # Hub path requires hubs and links
    for (i, j, k, l) in cand:
        m.addConstr(y[i, j, k, l] <= h[k])
        m.addConstr(y[i, j, k, l] <= h[l])
        m.addConstr(y[i, j, k, l] <= z[k, l])
    
    # Direct links only between non-hubs
    for i in nodes:
        for j in nodes:
            if i == j:
                continue
            m.addConstr(s[i, j] + h[i] <= 1.0)
            m.addConstr(s[i, j] + h[j] <= 1.0)
    
    # Flow conservation
    Wtot = float(W.sum())
    for i in nodes:
        for k in nodes:
            outflow = gp.quicksum(f[i, k, l] for l in nodes if l != k)
            inflow = gp.quicksum(f[i, l, k] for l in nodes if l != k)
            used_out = gp.quicksum(W[i, j] * y[i, j, k, l] for j in nodes for l in nodes if (i, j, k, l) in y)
            used_in = gp.quicksum(W[i, j] * y[i, j, l, k] for j in nodes for l in nodes if (i, j, l, k) in y)
            m.addConstr(inflow + used_out == outflow + used_in)
    
    # Link capacity constraints
    for i in nodes:
        for k in nodes:
            for l in nodes:
                if k == l:
                    continue
                m.addConstr(f[i, k, l] <= Wtot * z[k, l])
    
    # Apply warm-start if provided
    if init_solution is not None:
        if 'h' in init_solution:
            for k in nodes:
                h[k].start = float(init_solution['h'][k])
        
        if 'z' in init_solution:
            for (k, l), var in z.items():
                if k == l:
                    var.start = 0.0
                else:
                    var.start = float(init_solution['z'][k, l])
        
        if 'y' in init_solution:
            for key, var in y.items():
                i, j, k, l = key
                var.start = float(init_solution['y'].get((i, j, k, l), 0.0))
        
        if 's' in init_solution:
            for i in nodes:
                for j in nodes:
                    if i == j:
                        continue
                    s[i, j].start = float(init_solution['s'].get((i, j), 0.0))
        
        if 'f' in init_solution:
            for i in nodes:
                for k in nodes:
                    for l in nodes:
                        f[i, k, l].start = float(init_solution['f'].get((i, k, l), 0.0))
    
    variables = {
        'h': h, 'z': z, 'y': y, 's': s, 'f': f,
        'cand': cand, 'model': m
    }
    
    return m, variables


In [ ]:
# Cell 11: MA-DC Model with Hub Capacity Constraints
def build_ma_dc_model_with_capacity(W, C, alpha, R, F, G, Q, hub_capacity, init_solution=None):
    """
    Build MA-DC model with hub capacity constraints.
    
    Args:
        hub_capacity: list of capacity limits for each hub
        init_solution: optional warm-start solution
    """
    m, variables = build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q, init_solution)
    
    N = C.shape[0]
    nodes = range(N)
    h = variables['h']
    y = variables['y']
    cand = variables['cand']
    
    # Add hub capacity constraints
    for k in nodes:
        # Total demand assigned to hub k (as origin or destination)
        total_demand = gp.quicksum(W[i, j] * y[i, j, k, l] 
                                  for (i, j, k, l) in cand)
        total_demand += gp.quicksum(W[i, j] * y[i, j, l, k] 
                                   for (i, j, l, k) in cand)
        m.addConstr(total_demand <= hub_capacity[k] * h[k])
    
    return m, variables


In [ ]:
# Cell 12: Experiment Driver Functions
def run_cold_vs_warm(W, C, alpha, R, F, G, Q, timelimit=300, mipgap=1e-4):
    """
    Compare cold-start vs warm-start performance.
    """
    print("=== Cold vs Warm Start Comparison ===")
    
    # Cold start
    print("Running cold start...")
    t0 = time()
    m_cold, vars_cold = build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q)
    m_cold.Params.TimeLimit = timelimit
    m_cold.Params.MIPGap = mipgap
    m_cold.optimize()
    cold_time = time() - t0
    
    cold_results = {
        'obj': m_cold.ObjVal if m_cold.SolCount > 0 else None,
        'time': cold_time,
        'gap': m_cold.MIPGap if m_cold.SolCount > 0 else None,
        'nodes': m_cold.NodeCount
    }
    
    # Extract solution for warm-start
    if m_cold.SolCount > 0:
        init_solution = {
            'h': {k: vars_cold['h'][k].X for k in range(C.shape[0])},
            'z': {(k, l): vars_cold['z'][k, l].X for k in range(C.shape[0]) for l in range(C.shape[0])},
            'y': {(i, j, k, l): vars_cold['y'][i, j, k, l].X for (i, j, k, l) in vars_cold['cand']},
            's': {(i, j): vars_cold['s'][i, j].X for i in range(C.shape[0]) for j in range(C.shape[0]) if i != j},
            'f': {(i, k, l): vars_cold['f'][i, k, l].X for i in range(C.shape[0]) for k in range(C.shape[0]) for l in range(C.shape[0])}
        }
    else:
        init_solution = None
    
    # Warm start
    print("Running warm start...")
    t0 = time()
    m_warm, vars_warm = build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q, init_solution)
    m_warm.Params.TimeLimit = timelimit
    m_warm.Params.MIPGap = mipgap
    m_warm.optimize()
    warm_time = time() - t0
    
    warm_results = {
        'obj': m_warm.ObjVal if m_warm.SolCount > 0 else None,
        'time': warm_time,
        'gap': m_warm.MIPGap if m_warm.SolCount > 0 else None,
        'nodes': m_warm.NodeCount
    }
    
    # Comparison
    print(f"Cold start: obj={cold_results['obj']:.2f}, time={cold_results['time']:.2f}s")
    print(f"Warm start: obj={warm_results['obj']:.2f}, time={warm_results['time']:.2f}s")
    
    if cold_results['obj'] is not None and warm_results['obj'] is not None:
        time_improvement = (cold_results['time'] - warm_results['time']) / cold_results['time'] * 100
        print(f"Time improvement: {time_improvement:.1f}%")
    
    return cold_results, warm_results

def run_grid_cold(alpha_values, q_scales, W, C, R, F, G, timelimit=180, mipgap=1e-4):
    """
    Run parameter grid using cold start.
    """
    results = []
    
    for alpha in alpha_values:
        for q_scale in q_scales:
            print(f"\n=== alpha={alpha}, q_scale={q_scale} (cold) ===")
            
            # Scale Q matrix
            Q = G * q_scale
            
            # Build and solve
            t0 = time()
            m, vars_dict = build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q)
            m.Params.TimeLimit = timelimit
            m.Params.MIPGap = mipgap
            m.optimize()
            solve_time = time() - t0
            
            # Extract results
            if m.SolCount > 0:
                obj = m.ObjVal
                gap = m.MIPGap
                nodes = m.NodeCount
                
                # Count open hubs
                open_hubs = sum(1 for k in range(C.shape[0]) if vars_dict['h'][k].X > 0.5)
                
                # Count served demand
                served_demand = 0.0
                for (i, j, k, l), var in vars_dict['y'].items():
                    if var.X > 0.5:
                        served_demand += W[i, j]
                for i in range(C.shape[0]):
                    for j in range(C.shape[0]):
                        if i != j and vars_dict['s'][i, j].X > 0.5:
                            served_demand += W[i, j]
                
                sat_pct = 100.0 * served_demand / W.sum()
            else:
                obj = gap = nodes = open_hubs = sat_pct = None
            
            result = {
                'alpha': alpha,
                'q_scale': q_scale,
                'obj': obj,
                'time': solve_time,
                'gap': gap,
                'nodes': nodes,
                'hub_count': open_hubs,
                'sat_pct': sat_pct
            }
            
            results.append(result)
            print(f"Obj: {obj:.2f}, Time: {solve_time:.2f}s, Hubs: {open_hubs}, Sat: {sat_pct:.1f}%")
    
    return pd.DataFrame(results)

def run_grid_warm(init_solutions, alpha_values, q_scales, W, C, R, F, G, timelimit=180, mipgap=1e-4):
    """
    Run parameter grid using warm start with provided initial solutions.
    """
    results = []
    
    for alpha in alpha_values:
        for q_scale in q_scales:
            print(f"\n=== alpha={alpha}, q_scale={q_scale} (warm) ===")
            
            # Get initial solution for this parameter combination
            init_key = f"alpha_{alpha}_q_{q_scale}"
            init_solution = init_solutions.get(init_key)
            
            # Scale Q matrix
            Q = G * q_scale
            
            # Build and solve with warm start
            t0 = time()
            m, vars_dict = build_ma_dc_model_warmstart(W, C, alpha, R, F, G, Q, init_solution)
            m.Params.TimeLimit = timelimit
            m.Params.MIPGap = mipgap
            m.optimize()
            solve_time = time() - t0
            
            # Extract results (same as cold start)
            if m.SolCount > 0:
                obj = m.ObjVal
                gap = m.MIPGap
                nodes = m.NodeCount
                open_hubs = sum(1 for k in range(C.shape[0]) if vars_dict['h'][k].X > 0.5)
                
                served_demand = 0.0
                for (i, j, k, l), var in vars_dict['y'].items():
                    if var.X > 0.5:
                        served_demand += W[i, j]
                for i in range(C.shape[0]):
                    for j in range(C.shape[0]):
                        if i != j and vars_dict['s'][i, j].X > 0.5:
                            served_demand += W[i, j]
                
                sat_pct = 100.0 * served_demand / W.sum()
            else:
                obj = gap = nodes = open_hubs = sat_pct = None
            
            result = {
                'alpha': alpha,
                'q_scale': q_scale,
                'obj': obj,
                'time': solve_time,
                'gap': gap,
                'nodes': nodes,
                'hub_count': open_hubs,
                'sat_pct': sat_pct
            }
            
            results.append(result)
            print(f"Obj: {obj:.2f}, Time: {solve_time:.2f}s, Hubs: {open_hubs}, Sat: {sat_pct:.1f}%")
    
    return pd.DataFrame(results)


In [ ]:
# Cell 13: Visualization and Reporting Functions
def plot_performance_comparison(cold_df, warm_df, save_path=None):
    """
    Create performance comparison plots.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Runtime comparison
    ax1 = axes[0, 0]
    for df, label, color in [(cold_df, 'Cold Start', 'red'), (warm_df, 'Warm Start', 'blue')]:
        ax1.scatter(df['alpha'], df['time'], alpha=0.7, label=label, color=color)
    ax1.set_xlabel('Alpha')
    ax1.set_ylabel('Runtime (seconds)')
    ax1.set_title('Runtime vs Alpha')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Objective comparison
    ax2 = axes[0, 1]
    for df, label, color in [(cold_df, 'Cold Start', 'red'), (warm_df, 'Warm Start', 'blue')]:
        ax2.scatter(df['q_scale'], df['obj'], alpha=0.7, label=label, color=color)
    ax2.set_xlabel('Q Scale')
    ax2.set_ylabel('Objective Value')
    ax2.set_title('Objective vs Q Scale')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Hub count comparison
    ax3 = axes[1, 0]
    for df, label, color in [(cold_df, 'Cold Start', 'red'), (warm_df, 'Warm Start', 'blue')]:
        ax3.scatter(df['q_scale'], df['hub_count'], alpha=0.7, label=label, color=color)
    ax3.set_xlabel('Q Scale')
    ax3.set_ylabel('Number of Hubs')
    ax3.set_title('Hub Count vs Q Scale')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Demand satisfaction
    ax4 = axes[1, 1]
    for df, label, color in [(cold_df, 'Cold Start', 'red'), (warm_df, 'Warm Start', 'blue')]:
        ax4.scatter(df['alpha'], df['sat_pct'], alpha=0.7, label=label, color=color)
    ax4.set_xlabel('Alpha')
    ax4.set_ylabel('Demand Satisfaction (%)')
    ax4.set_title('Demand Satisfaction vs Alpha')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()

def print_solution_summary(m, vars_dict, W, C):
    """
    Print detailed solution summary.
    """
    if m.SolCount == 0:
        print("No solution found!")
        return
    
    N = C.shape[0]
    
    # Open hubs
    open_hubs = [k for k in range(N) if vars_dict['h'][k].X > 0.5]
    print(f"\n=== SOLUTION SUMMARY ===")
    print(f"Open hubs: {open_hubs}")
    print(f"Number of hubs: {len(open_hubs)}")
    
    # Hub paths
    hub_paths = []
    for (i, j, k, l), var in vars_dict['y'].items():
        if var.X > 0.5:
            hub_paths.append((i, j, k, l, W[i, j]))
    
    print(f"Hub paths: {len(hub_paths)}")
    if hub_paths:
        print("Top 5 hub paths by demand:")
        sorted_paths = sorted(hub_paths, key=lambda x: x[4], reverse=True)[:5]
        for i, j, k, l, demand in sorted_paths:
            print(f"  {i}->{j} via {k}->{l}: {demand:.4f}")
    
    # Direct links
    direct_links = []
    for i in range(N):
        for j in range(N):
            if i != j and vars_dict['s'][i, j].X > 0.5:
                direct_links.append((i, j, W[i, j]))
    
    print(f"Direct links: {len(direct_links)}")
    if direct_links:
        print("Top 5 direct links by demand:")
        sorted_direct = sorted(direct_links, key=lambda x: x[2], reverse=True)[:5]
        for i, j, demand in sorted_direct:
            print(f"  {i}->{j}: {demand:.4f}")
    
    # Demand satisfaction
    total_served = sum(demand for _, _, _, _, demand in hub_paths) + sum(demand for _, _, demand in direct_links)
    total_demand = W.sum()
    print(f"Demand satisfaction: {total_served:.4f}/{total_demand:.4f} ({100*total_served/total_demand:.1f}%)")
    
    # Objective breakdown
    print(f"Objective value: {m.ObjVal:.2f}")

def save_results_to_csv(df, filename):
    """
    Save results DataFrame to CSV.
    """
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")


In [ ]:

# Cell 14: Main Experiment Execution
def run_comprehensive_experiment():
    """
    Run the complete MA-DC experiment suite.
    """
    print("=== MA-DC HUB LOCATION COMPREHENSIVE EXPERIMENT ===")
    
    # Use existing scaled parameters
    alpha_values = [0.2, 0.35, 0.5]
    q_scales = [1.0, 0.5, 0.2]
    
    # Get parameters using existing functions
    R, F, G, Q, _ = cab_params("high", "low", 0.2)
    F_scaled = F * tau * 0.30  # Use gamma=0.30 as in existing code
    G_scaled = G * tau * 0.30
    
    print(f"Parameters: N={N}, alpha_values={alpha_values}, q_scales={q_scales}")
    
    # Run cold start grid
    print("\n1. Running cold start parameter grid...")
    cold_results = run_grid_cold(alpha_values, q_scales, W, C_scaled, R, F_scaled, G_scaled)
    
    # Create initial solutions dictionary for warm start
    init_solutions = {}
    for _, row in cold_results.iterrows():
        if row['obj'] is not None:  # Only if solution was found
            key = f"alpha_{row['alpha']}_q_{row['q_scale']}"
            # For simplicity, we'll use the cold start solution as warm start
            # In practice, you might want to store the actual solution vectors
            init_solutions[key] = None  # Placeholder for actual solution
    
    # Run warm start grid
    print("\n2. Running warm start parameter grid...")
    warm_results = run_grid_warm(init_solutions, alpha_values, q_scales, W, C_scaled, R, F_scaled, G_scaled)
    
    # Performance comparison
    print("\n3. Creating performance comparison plots...")
    plot_performance_comparison(cold_results, warm_results, "ma_dc_performance_comparison.png")
    
    # Save results
    print("\n4. Saving results...")
    save_results_to_csv(cold_results, "ma_dc_cold_results.csv")
    save_results_to_csv(warm_results, "ma_dc_warm_results.csv")
    
    # Run cold vs warm comparison for one parameter set
    print("\n5. Running detailed cold vs warm comparison...")
    cold_vs_warm = run_cold_vs_warm(W, C_scaled, 0.2, R, F_scaled, G_scaled, G_scaled * 0.5)
    
    # Test hub capacity extension
    print("\n6. Testing hub capacity extension...")
    hub_capacity = [0.1] * N  # 10% capacity for each hub
    m_cap, vars_cap = build_ma_dc_model_with_capacity(W, C_scaled, 0.2, R, F_scaled, G_scaled, G_scaled * 0.5, hub_capacity)
    m_cap.Params.TimeLimit = 300
    m_cap.Params.MIPGap = 1e-4
    m_cap.optimize()
    
    if m_cap.SolCount > 0:
        print(f"Capacity-constrained solution: obj={m_cap.ObjVal:.2f}")
        print_solution_summary(m_cap, vars_cap, W, C_scaled)
    
    print("\n=== EXPERIMENT COMPLETE ===")
    
    return cold_results, warm_results, cold_vs_warm

## Run the comprehensive experiment
cold_df, warm_df, comparison = run_comprehensive_experiment()

## Print summary statistics
print("\n=== SUMMARY STATISTICS ===")
print(f"Cold start average time: {cold_df['time'].mean():.2f}s")
print(f"Warm start average time: {warm_df['time'].mean():.2f}s")
print(f"Time improvement: {((cold_df['time'].mean() - warm_df['time'].mean()) / cold_df['time'].mean() * 100):.1f}%")
print(f"Cold start average objective: {cold_df['obj'].mean():.2f}")
print(f"Warm start average objective: {warm_df['obj'].mean():.2f}")
